## Dependencies
***

In [25]:
#--Dependencies--#

#-- Data Cleaning Libraries:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

#-- Data Visualization Libraries:
from matplotlib import pyplot as plt
import seaborn as sns #--just in case we need it, probably won't


#-- Web Scraping Libraries:
import os
import time
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser


***

## Web Scraping 
***

In [41]:
#Settings for accessing Website

executable_path = {"executable_path": "chromedriver.exe"}

browser = Browser ('chrome', **executable_path, headless=False)



### Defining Functions for Data Scraping and Cleaning
***

#### NBA
***

In [86]:
#Pre-defining Variables for NBA scrape

#Empty DataFrame for NBA
NBA_game_df = pd.DataFrame()


#Reference to names
NBA_refer = "basketball-reference"

#URL
NBA_url = "https://www.basketball-reference.com"

#Create Empty List for years we want to scrape for
years = ["1990", "1991", "1992", "1993", "1994", "1995",\
         "1996", "1997", "1998", "1999", "2000","2001", \
         "2002", "2003", "2004", "2005", "2006", "2007",\
         "2008", "2009", "2010", "2011", "2012", "2013",\
         "2014", "2015", "2016"]



In [93]:
#Define our function for scraping NBA Data---> ** need to come back to this and finish it off
def scrape_nba_data(page_url):
    #URL's for both NBA and NCAA
    #Reference global variable: NBA_game_df
    global NBA_game_df
    
    for year in years:

        #Set the rest of the url
        url = f"https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html"

        #Visit the NBA Web page
        browser.visit(url)

        #Retrieve the html for the web page
        html = browser.html

        #Use pandas to read html
        year_html = pd.read_html(html, header = 0)

        #Get the second Table with all of the Data
        year_html = year_html[1]

        #Convert into DataFrame
        year_df = pd.DataFrame(year_html)

        #Delete Column of Ranking: "Rk"
        year_df = year_df.rename(columns=({"Rk" : "Year"}))

        #Apply the year to the Year column for each row
        year_df["Year"] = year_df["Year"].apply(lambda x: year)



        #Append to main DataFrame: NBA_game_df
        if NBA_game_df.empty: 
            NBA_game_df = year_df
        else: 
            NBA_game_df = NBA_game_df.append(year_df, ignore_index = True)

        #hoooldd on wait a second, let me put some sleep in it
        time.sleep(1)



In [12]:
#Function for cleaning and merging NBA data

def clean_nba_data (page_html):
    # Calculate PER
    
    # Set Unique ID to players based upon their name



#### NCAA
***

In [80]:
#Predefining variables for NCAA scrape

NCAA_df = pd.DataFrame()
NCAA_refer = "sports-reference"
NCAA_url = "https://www.sports-reference.com/cbb/play-index/psl_finder.cgi?request=1&match=single&year_min=1993&year_max=2016&class_is_fr=Y&class_is_so=Y&class_is_jr=Y&class_is_sr=Y&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&games_type=A&c1stat=fga&c1comp=gt&order_by=year_id&order_by_asc=Y"

In [81]:
#Function for scraping NCAA data

def scrape_ncaa_data(page_url):
    global NCAA_df
    url = page_url
    browser.visit(url)
    click_count = 0
    
    while NCAA_refer in url:
        try:
            if click_count == 0:

                #Get html of web page
                html = browser.html

                #Read html using pandas
                read_html = pd.read_html(html)

                #Create DF and append to ncaa_df
                page_df = pd.DataFrame(read_html[1])
                NCAA_df = page_df

                #Add to click counter
                click_count += 1
                
                #Sleep
                time.sleep(1)
                
                #click to go to next page
                next_page = browser.find_link_by_text("Next page")
                next_page.click()
                


            else:
                #Get html of web page
                html = browser.html

                #Read html using pandas
                read_html = pd.read_html(html)

                #print(read_html[1])
                page_df = pd.DataFrame(read_html[1])
                NCAA_df = NCAA_df.append(page_df, ignore_index= True)

                #hoooldd on wait a second, let me put some sleep in it
                time.sleep(1)
            
                #click to go to next page
                next_page = browser.find_link_by_text("Next page")
                next_page.click()
        except AttributeError as error:
            print("End of scrape")
            window.close()
            

In [83]:
#Function for cleaning and merging NCAA data ** Will work on this later

def clean_ncaa_data(ncaa_csv):
    #Calculate PER

SyntaxError: unexpected EOF while parsing (<ipython-input-83-87bced7b6067>, line 4)

### Scraping for Data
***

#### NCAA
***




In [89]:
#Scrapety scrape scrape
scrape_ncaa_data(NCAA_url)

KeyboardInterrupt: 

In [65]:
#Test print
NCAA_df
NCAA_copy = NCAA_df

#Confirm last value on Web Page is equal to that of DF
NCAA_df.tail()


,Unnamed: 0_level_0,Totals,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,...,Unnamed: 16_level_0,Unnamed: 17_level_0,Unnamed: 18_level_0,Unnamed: 19_level_0,Unnamed: 20_level_0,Unnamed: 21_level_0,Unnamed: 22_level_0,Unnamed: 23_level_0,Unnamed: 24_level_0,Unnamed: 25_level_0
,Rk,Player,Class,Season,Pos,School,Conf,G,MP,FG,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
112161,103854,Stephen Zimmerman Jr.,FR,2015-16,F,Nevada-Las Vegas,MWC,26,680,102,...,101,56,171,227,22,14,51,54,79,272
112162,103855,Vinny Zollo,SR,2015-16,F,Cleveland State,Horizon,29,642,69,...,48,52,112,164,32,14,12,44,74,194
112163,103856,Nisre Zouzoua,FR,2015-16,G,Bryant,NEC,31,939,131,...,89,20,78,98,44,25,8,45,86,397
112164,103857,Jake Zuilhof,SR,2015-16,F,Central Arkansas,Southland,28,710,69,...,76,66,121,187,18,10,37,51,102,177
112165,103858,Johnny Zuppardo,SR,2015-16,F,Mississippi State,SEC,31,358,38,...,6,16,52,68,7,14,5,13,60,98


In [60]:
#Push Raw Data to CSV (So we don't have to scrape again)

NCAA_copy.to_csv("NCAA_raw.csv", index = False)

#### NBA
***

In [94]:
#Scrapety scrape scrape

scrape_nba_data(NBA_url)

In [96]:
#Test print
NBA_game_df

NBA_copy = NBA_game_df

#Confirm last record on Web Page with that on DF
NBA_copy.tail()

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
16035,2016,Joe Young,PG,23,IND,41,0,9.4,1.5,4.1,...,.800,0.1,1.1,1.2,1.6,0.4,0.0,0.8,0.7,3.8
16036,2016,Nick Young,SG,30,LAL,54,2,19.1,2.3,6.9,...,.829,0.3,1.5,1.8,0.6,0.4,0.1,0.6,0.9,7.3
16037,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,...,.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1
16038,2016,Cody Zeller,C,23,CHO,73,60,24.3,3.2,6.0,...,.754,1.9,4.3,6.2,1.0,0.8,0.9,0.9,2.8,8.7
16039,2016,Tyler Zeller,C,26,BOS,60,3,11.8,2.3,4.8,...,.815,1.0,1.9,3.0,0.5,0.2,0.4,0.8,1.6,6.1


In [97]:
#Push Raw Data to csv
NBA_copy.to_csv("NBA_raw.csv", index = False)



### Cleaning Data
***

In [98]:
#Set PER calculation function

def per (row):
    try:
        calc = ((int(row.FG) * 85.910) + (int(row.STL) * 53.897) + (int(row["3P"]) * 51.757)\
               + (int(row.FT) * 46.845) + (int(row.BLK) * 39.190) + (int(row.ORB) * 39.190)\
               + (int(row.AST) * 34.677) + (int(row.DRB) * 14.707) + (int(row.PF) * 17.174)\
               + ((int(row.FTA) - int(row.FT)) * 20.091) + (int(row.TOV) * 53.897)) * (1/int(row.MP))
        return round(calc, 2)
    
    except ZeroDivisionError as zeroerror:
        calc = ((int(row.FG) * 85.910) + (int(row.STL) * 53.897) + (int(row["3P"]) * 51.757)\
               + (int(row.FT) * 46.845) + (int(row.BLK) * 39.190) + (int(row.ORB) * 39.190)\
               + (int(row.AST) * 34.677) + (int(row.DRB) * 14.707) + (int(row.PF) * 17.174)\
               + ((int(row.FTA) - int(row.FT)) * 20.091) + (int(row.TOV) * 53.897)) * (0) 
        return round(calc, 2)
    except ValueError as error: 
            calc = ((float(row.FG) * 85.910) + (float(row.STL) * 53.897) + (float(row["3P"]) * 51.757)\
               + (float(row.FT) * 46.845) + (float(row.BLK) * 39.190) + (float(row.ORB) * 39.190)\
               + (float(row.AST) * 34.677) + (float(row.DRB) * 14.707) + (float(row.PF) * 17.174)\
               + ((float(row.FTA) - float(row.FT)) * 20.091) + (float(row.TOV) * 53.897)) * (1/float(row.MP)) 
            return round(calc, 2)

In [13]:
#Convert Seasons into a single year value

def normalize_year(y):
    try:
        return (y[0:4])# + y[-1])
    except TypeError as error:
        pass
#Test
NCAA_df.Season.head().apply(lambda x: normalize_year(x))

1    1992
2    1992
3    1992
4    1992
5    1992
Name: Season, dtype: object

#### NCAA
***

In [233]:
#Note: Data has not been collected for per game stats, will need to calculate per game stats.

#Read in CSV to be cleaned: "NCAA_raw.csv"
NCAA_df = pd.read_csv("NCAA_raw.csv")

NCAA_df.head()



,Unnamed: 0_level_0,Totals,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,...,Unnamed: 16_level_0,Unnamed: 17_level_0,Unnamed: 18_level_0,Unnamed: 19_level_0,Unnamed: 20_level_0,Unnamed: 21_level_0,Unnamed: 22_level_0,Unnamed: 23_level_0,Unnamed: 24_level_0,Unnamed: 25_level_0
0,Rk,Player,Class,Season,Pos,School,Conf,G,MP,FG,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,1,Craig Aamot,SO,1992-93,G,Marquette,GMWC,20,NaN,6,...,12,NaN,NaN,22,14,1,0,NaN,NaN,24
2,2,Kahhar Abdul-Wali,FR,1992-93,G,Coppin State,MEAC,9,NaN,0,...,2,NaN,NaN,2,0,0,0,NaN,NaN,0
3,3,Abdul Abdullah,JR,1992-93,G,Providence,Big East,32,822,18,...,62,NaN,NaN,55,181,30,6,NaN,NaN,69
4,4,Shawn Abercrombie,SO,1992-93,G,St. Bonaventure,A-10,1,NaN,0,...,0,NaN,NaN,0,2,1,0,NaN,NaN,0


In [234]:
#Set column names 

ncaa_cols =["Rk", "Player", "Class", "Season", "Pos", "School", "Conf", "G", "MP", "FG", "FGA", "2P", "2PA", "3P",\
            "3PA", "FT", "FTA", "ORB", "DRB", "TRB", "AST", "STL","BLK", "TOV", "PF", "PTS"]


#Set Columns for NCAA DF
NCAA_df.columns = ncaa_cols

NCAA_df = NCAA_df[NCAA_df.Season != "Season"]
NCAA_df = NCAA_df[NCAA_df.Player != "Totals"]

#Test print
NCAA_df.head()



,Rk,Player,Class,Season,Pos,School,Conf,G,MP,FG,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,1,Craig Aamot,SO,1992-93,G,Marquette,GMWC,20,NaN,6,...,12,NaN,NaN,22,14,1,0,NaN,NaN,24
2,2,Kahhar Abdul-Wali,FR,1992-93,G,Coppin State,MEAC,9,NaN,0,...,2,NaN,NaN,2,0,0,0,NaN,NaN,0
3,3,Abdul Abdullah,JR,1992-93,G,Providence,Big East,32,822,18,...,62,NaN,NaN,55,181,30,6,NaN,NaN,69
4,4,Shawn Abercrombie,SO,1992-93,G,St. Bonaventure,A-10,1,NaN,0,...,0,NaN,NaN,0,2,1,0,NaN,NaN,0
5,5,Frank Ableson,JR,1992-93,G,Cornell,Ivy,26,NaN,34,...,33,NaN,NaN,29,50,19,3,NaN,NaN,114


In [235]:
#Remove Rk column from our lives, good riddance

NCAA_df = NCAA_df.drop("Rk", axis = 1)

#Test print
NCAA_df.head()


,Player,Class,Season,Pos,School,Conf,G,MP,FG,FGA,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,Craig Aamot,SO,1992-93,G,Marquette,GMWC,20,NaN,6,16,...,12,NaN,NaN,22,14,1,0,NaN,NaN,24
2,Kahhar Abdul-Wali,FR,1992-93,G,Coppin State,MEAC,9,NaN,0,7,...,2,NaN,NaN,2,0,0,0,NaN,NaN,0
3,Abdul Abdullah,JR,1992-93,G,Providence,Big East,32,822,18,75,...,62,NaN,NaN,55,181,30,6,NaN,NaN,69
4,Shawn Abercrombie,SO,1992-93,G,St. Bonaventure,A-10,1,NaN,0,0,...,0,NaN,NaN,0,2,1,0,NaN,NaN,0
5,Frank Ableson,JR,1992-93,G,Cornell,Ivy,26,NaN,34,95,...,33,NaN,NaN,29,50,19,3,NaN,NaN,114


In [236]:
#Fill NaN values with 0

NCAA_df = NCAA_df.fillna(0)

#Test print
NCAA_df.head()



,Player,Class,Season,Pos,School,Conf,G,MP,FG,FGA,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,Craig Aamot,SO,1992-93,G,Marquette,GMWC,20,0,6,16,...,12,0,0,22,14,1,0,0,0,24
2,Kahhar Abdul-Wali,FR,1992-93,G,Coppin State,MEAC,9,0,0,7,...,2,0,0,2,0,0,0,0,0,0
3,Abdul Abdullah,JR,1992-93,G,Providence,Big East,32,822,18,75,...,62,0,0,55,181,30,6,0,0,69
4,Shawn Abercrombie,SO,1992-93,G,St. Bonaventure,A-10,1,0,0,0,...,0,0,0,0,2,1,0,0,0,0
5,Frank Ableson,JR,1992-93,G,Cornell,Ivy,26,0,34,95,...,33,0,0,29,50,19,3,0,0,114


In [255]:
#make new df to store MP of 0, Filter NCAA_df to include values of MP > 0


zero_MP_df = NCAA_df[(NCAA_df["MP"] == 0)]

NCAA_df = NCAA_df[NCAA_df["MP"] != 0]

NCAA_df.tail()

,Player,Class,Season,Pos,School,Conf,G,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
112162,Stephen Zimmerman Jr.,FR,2015-16,F,Nevada-Las Vegas,MWC,26,680,102,214,...,56,171,227,22,14,51,54,79,272,37.10
112163,Vinny Zollo,SR,2015-16,F,Cleveland State,Horizon,29,642,69,191,...,52,112,164,32,14,12,44,74,194,28.98
112164,Nisre Zouzoua,FR,2015-16,G,Bryant,NEC,31,939,131,321,...,20,78,98,44,25,8,45,86,397,28.94
112165,Jake Zuilhof,SR,2015-16,F,Central Arkansas,Southland,28,710,69,122,...,66,121,187,18,10,37,51,102,177,28.14
112166,Johnny Zuppardo,SR,2015-16,F,Mississippi State,SEC,31,358,38,93,...,16,52,68,7,14,5,13,60,98,24.41


In [261]:
#-Convertr columns into ints and divide by Games played column-#
#Convert Games Played column to integer
NCAA_df["G"] = pd.to_numeric(NCAA_df["G"])

#Get Stats columns from DF (excluding Games Played)
stats_cols = NCAA_df.columns.values[7:]

# #loop to check if they are a string, if true convert it to integer
for series in stats_cols:
    string_check = is_string_dtype(NCAA_df[series])
    num_check = is_numeric_dtype(NCAA_df[series])
    
    if string_check == True:
        print(f"The column {series} is a string: {string_check}")
        NCAA_df[series] = pd.to_numeric(NCAA_df[series])
        NCAA_df[series] = round((NCAA_df[series]/NCAA_df.G), 3)
     
    
    else:
        print(f"The column {series} is a numeric: {num_check}")
        NCAA_df[series] = pd.to_numeric(NCAA_df[series])
        NCAA_df[series] = round((NCAA_df[series]/NCAA_df.G), 3)


The column MP is a numeric: True
The column FG is a numeric: True
The column FGA is a numeric: True
The column 2P is a numeric: True
The column 2PA is a numeric: True
The column 3P is a numeric: True
The column 3PA is a numeric: True
The column FT is a numeric: True
The column FTA is a numeric: True
The column ORB is a numeric: True
The column DRB is a numeric: True
The column TRB is a numeric: True
The column AST is a numeric: True
The column STL is a numeric: True
The column BLK is a numeric: True
The column TOV is a numeric: True
The column PF is a numeric: True
The column PTS is a numeric: True
The column PER is a numeric: True


In [257]:
#Make PER column and set values by passing 'per' function
NCAA_df["PER"] = NCAA_df.apply(lambda row : per(row), axis = 1)
NCAA_df.head()

,Player,Class,Season,Pos,School,Conf,G,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
3,Abdul Abdullah,JR,1992-93,G,Providence,Big East,32,25.688,0.562,2.344,...,0.0,0.0,1.719,5.656,0.938,0.188,0.000,0.000,2.156,7.74
18,Mark Alberts,SR,1992-93,G,Akron,MAC,26,37.154,5.385,13.500,...,0.0,0.0,1.808,3.077,0.885,0.000,2.000,1.308,17.077,25.93
26,Chris Alexander,FR,1992-93,C,Virginia,ACC,13,3.462,0.154,0.308,...,0.0,0.0,1.000,0.154,0.000,0.462,0.154,0.846,0.538,6.70
27,Cory Alexander,SO,1992-93,G,Virginia,ACC,31,36.065,6.871,15.161,...,0.0,0.0,3.452,4.645,0.839,0.226,3.419,2.419,18.806,30.95
31,Shon Alexander,FR,1992-93,G,Oklahoma,Big 8,20,6.950,0.400,1.200,...,0.0,0.0,1.000,0.300,0.550,0.100,0.450,1.050,1.300,2.86


In [259]:
#Convert for Entire series

NCAA_df.Season = NCAA_df.Season.apply(lambda x: normalize_year(x))

#Test Print
NCAA_df.head()
NCAA_df.tail()

NCAA_df.sort_values(by = "PTS", ascending=False)

,Player,Class,Season,Pos,School,Conf,G,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
30929,John Crider,0,1999,0,Kansas,Big 12,0,inf,NaN,inf,...,NaN,NaN,inf,inf,inf,NaN,inf,inf,inf,NaN
21158,Bubba Wells,SR,1996,F,Austin Peay,OVC,19,36.053000,11.474,22.053000,...,0.000,0.000,7.105000,2.211000,1.316000,0.632,3.105000,2.895000,31.737000,46.92
7456,Glenn Robinson,JR,1993,F,Purdue,Big Ten,34,34.294000,10.824,22.412000,...,0.000,0.000,10.118000,1.941000,1.647000,0.912,4.088000,2.588000,30.294000,47.13
19088,Charles Jones,JR,1996,G,Long Island University,NEC,30,37.300000,11.267,25.000000,...,0.000,0.000,6.000000,4.633000,2.333000,0.233,0.000000,0.000000,30.100000,41.83
39920,Jason Conley,FR,2001,G,Virginia Military Institute,Southern,28,34.250000,10.179,21.786000,...,2.857,5.143,8.000000,2.393000,2.929000,0.393,3.679000,1.857000,29.286000,52.11
3128,Isaiah Rider,SR,1992,G,Nevada-Las Vegas,Big West,28,35.429000,10.071,19.571000,...,0.000,0.000,8.929000,2.536000,1.464000,0.643,0.000000,0.000000,29.071000,38.72
12228,Kurt Thomas,SR,1994,C,Texas Christian,SWC,27,32.296000,10.667,19.481000,...,0.000,0.000,14.556000,1.185000,1.889000,2.444,2.852000,3.926000,28.926000,49.17
83194,Jimmer Fredette,SR,2010,G,Brigham Young,MWC,37,35.757000,9.351,20.676000,...,0.649,2.784,3.432000,4.324000,1.324000,0.027,3.541000,1.324000,28.865000,46.59
73023,Stephen Curry,JR,2008,G,Davidson,Southern,34,33.676000,9.176,20.206000,...,0.618,3.824,4.441000,5.559000,2.529000,0.235,3.706000,2.382000,28.647000,53.06
159,Vin Baker,SR,1992,F,Hartford,NAC,28,36.393000,10.893,22.821000,...,0.000,0.000,10.714000,1.929000,1.393000,2.643,2.893000,2.821000,28.286000,42.07


#### NBA
***

In [169]:
#Note: Data has been collected for Per Game Stats

#Read in CSV to be cleaned: "NBA_raw.csv"
NBA_df = pd.read_csv("NBA_raw.csv")

NBA_df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1990,Mark Acres,C,27,ORL,80,50,21.1,1.7,3.6,...,.692,1.9,3.5,5.4,0.8,0.5,0.3,0.9,3.1,4.5
1,1990,Michael Adams,PG,27,DEN,79,74,34.1,5.0,12.5,...,.850,0.6,2.2,2.8,6.3,1.5,0.0,1.8,1.7,15.5
2,1990,Mark Aguirre,SF,30,DET,78,40,25.7,5.6,11.5,...,.756,1.5,2.4,3.9,1.9,0.4,0.2,1.6,2.6,14.1
3,1990,Danny Ainge,PG,30,SAC,75,68,36.4,6.7,15.4,...,.831,0.9,3.4,4.3,6.0,1.5,0.2,2.5,3.2,17.9
4,1990,Mark Alarie,PF,26,WSB,82,10,23.1,4.5,9.6,...,.812,1.8,2.7,4.6,1.7,0.7,0.5,1.2,2.7,10.5


In [172]:
#Fill NaN values with 0

NBA_df = NBA_df.fillna(0)


In [211]:
#Set PER calculation function
NBA_df["PER"] = NBA_df.apply(lambda row : per(row), axis = 1)

ValueError: ("invalid literal for int() with base 10: '1.7'", 'occurred at index 0')